# Load ORM

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

In [2]:
engine = create_engine('sqlite:///db/mdb.db')

# Load Models

In [3]:
Base = declarative_base(name="Model")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [4]:
class Region(Base):
    __tablename__ = "region"
    id = Column(Integer, primary_key=True)
    department_code = Column(Integer)
    department_name = Column(String)

In [5]:
class Parti(Base):
    __tablename__ = "parti"
    id = Column(Integer, primary_key=True)
    parti_name = Column(String)
    candidat = relationship("CandidatParti", back_populates="parti")

In [6]:
class Candidat(Base):
    __tablename__ = "candidat"
    id = Column(Integer, primary_key=True)
    candidat_name = Column(String)
    parti = relationship("CandidatParti", back_populates="candidat")


In [7]:
class CandidatParti(Base):
    __tablename__ = "candidat_parti"
    id = Column(Integer, primary_key=True)
    candidat_id = Column(Integer, ForeignKey('candidat.id'))
    parti_id = Column(Integer, ForeignKey('parti.id'))
    candidat = relationship("Candidat")
    parti = relationship("Parti")

In [8]:
class UrneVote(Base):
    __tablename__ = "urne_vote"
    id = Column(Integer, primary_key=True)
    circonscription = Column(Integer)
    region_id = Column(Integer, ForeignKey('region.id'))
    annee = Column(String)
    final_round = Column(Boolean, default=False)

In [9]:
class ResultatCondidatParti(Base):
    __tablename__ = "resultat_candidat"
    id = Column(Integer, primary_key=True)
    urne_vote_id = Column(Integer, ForeignKey('urne_vote.id'))  
    candidat_parti = Column(Integer, ForeignKey('candidat_parti.id'))  
    value = Column(Integer, default=0)

In [10]:
class ResultatMetaInfo(Base):
    __tablename__ = "resultat_metainfo"
    id = Column(Integer, primary_key=True)
    urne_vote_id = Column(Integer, ForeignKey('urne_vote.id'))  
    inscripts = Column(Integer, default=0)
    votants = Column(Integer, default=0)
    nullparts = Column(Integer, default=0)
    exprimes = Column(Integer, default=0)

## Create Schema

In [11]:
Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)

# Load CSVs

In [12]:
from glob import glob
import pandas as ps
import os

In [13]:
files = glob("presi/*.csv")

## Processing

## Helpers

In [14]:
def findCandidatOrCreate(candidat_name):
    candidat = Candidat(candidat_name = candidat_name)
    candidats = (
        session
        .query(Candidat)
        .filter(Candidat.candidat_name == candidat_name)
        .all()
    )
    

In [15]:
def extractCandidatPart(str):
    sp = str.split(" (")
    if(len(sp) > 1):
        return (sp[0], sp[1].replace(")", ""))
    else:
        return (sp[0], "NEUTRE")

## First Round

In [16]:
def firstTerm(df):
    candidats = [extractCandidatPart(x) for x in df.columns if x.isupper()]
    for (candidat_name, parti_name) in candidats:
        candidat = Candidat(candidat_name = candidat_name)
        session.add(candidat)
        session.commit()

## Second Round

In [17]:
def secondTerm(df):
    pass

## Main Proccess

In [18]:
for file in files:
    df = ps.read_csv(file)
    _, tail = os.path.split(file)
    if tail.find("t1_circ") == -1:
        secondTerm(df)
    else:
        firstTerm(df)